In [5]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [13]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2
housing = fetch_california_housing()
m,n = housing.data.shape
housing_plus_bias = np.c_[np.ones([m,1]),housing.data]
X_data = tf.constant(housing_plus_bias)
y_data = tf.constant(housing.target.reshape([-1,1]))

In [3]:
sess = tf.Session()
sess.run(x.initializer) #in the future, we can use global initializer
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)
# the session is automatically closed at the end of the with block

42


In [5]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [6]:
# if you don't want the with block, you can use interactive session
# it sets itself as the default session, so you can do init.run() instead of sess.run(init)
# but you have to close it at the end
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)


42


In [7]:
# any node you create is automatically added to the default graph
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [8]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


In [9]:
tf.reset_default_graph()

In [10]:
z = tf.Variable(1)
tf.get_default_graph().get_operations()

[<tf.Operation 'Variable/initial_value' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>]

In [11]:
tf.reset_default_graph()
tf.get_default_graph().get_operations()

[]

In [13]:
housing = fetch_california_housing()
m,n = housing.data.shape
housing_plus_bias = np.c_[np.ones([m,1]),housing.data]
X = tf.constant(housing_plus_bias)
y = tf.constant(housing.target.reshape([-1,1]))
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)
with tf.Session() as sess:
    theta_value = theta.eval()

In [14]:
theta.shape

TensorShape([Dimension(9), Dimension(1)])

In [7]:
# scale data for gradient descent
scaler = StandardScaler()
X_scaled = scaler.fit_transform(housing.data)
X_scaled_plus_bias = np.c_[np.ones([m,1]), X_scaled]

In [19]:
# manual gradient descent
tf.reset_default_graph()
n_epochs = 1000
learning_rate = 0.01
X = tf.constant(X_scaled_plus_bias, dtype=tf.float32)
y = tf.constant(housing.target.reshape([-1,1]), dtype=tf.float32)
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0))
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error))
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate*gradients)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 50 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 2.86821
Epoch 50 MSE = 0.887818
Epoch 100 MSE = 0.609831
Epoch 150 MSE = 0.561184
Epoch 200 MSE = 0.549296
Epoch 250 MSE = 0.544395
Epoch 300 MSE = 0.541257
Epoch 350 MSE = 0.538806
Epoch 400 MSE = 0.536765
Epoch 450 MSE = 0.535031
Epoch 500 MSE = 0.533549
Epoch 550 MSE = 0.53228
Epoch 600 MSE = 0.531192
Epoch 650 MSE = 0.530258
Epoch 700 MSE = 0.529456
Epoch 750 MSE = 0.528767
Epoch 800 MSE = 0.528174
Epoch 850 MSE = 0.527664
Epoch 900 MSE = 0.527225
Epoch 950 MSE = 0.526846


In [20]:
# auto diff gradient descent
tf.reset_default_graph()
n_epochs = 1000
learning_rate = 0.01
X = tf.constant(X_scaled_plus_bias, dtype=tf.float32)
y = tf.constant(housing.target.reshape([-1,1]), dtype=tf.float32)
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0))
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error))
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate*gradients)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 50 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 8.63968
Epoch 50 MSE = 1.3958
Epoch 100 MSE = 0.741987
Epoch 150 MSE = 0.646694
Epoch 200 MSE = 0.622188
Epoch 250 MSE = 0.608608
Epoch 300 MSE = 0.597865
Epoch 350 MSE = 0.588689
Epoch 400 MSE = 0.580746
Epoch 450 MSE = 0.573849
Epoch 500 MSE = 0.567852
Epoch 550 MSE = 0.56263
Epoch 600 MSE = 0.558079
Epoch 650 MSE = 0.554107
Epoch 700 MSE = 0.550637
Epoch 750 MSE = 0.5476
Epoch 800 MSE = 0.544941
Epoch 850 MSE = 0.542608
Epoch 900 MSE = 0.540559
Epoch 950 MSE = 0.538757


In [23]:
# optimizer gradient descent
tf.reset_default_graph()
n_epochs = 1000
learning_rate = 0.01
X = tf.constant(X_scaled_plus_bias, dtype=tf.float32)
y = tf.constant(housing.target.reshape([-1,1]), dtype=tf.float32)
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0))
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 50 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 8.81921
Epoch 50 MSE = 1.57893
Epoch 100 MSE = 0.957332
Epoch 150 MSE = 0.834873
Epoch 200 MSE = 0.77963
Epoch 250 MSE = 0.739735
Epoch 300 MSE = 0.707001
Epoch 350 MSE = 0.679416
Epoch 400 MSE = 0.656033
Epoch 450 MSE = 0.636184
Epoch 500 MSE = 0.619328
Epoch 550 MSE = 0.605013
Epoch 600 MSE = 0.592855
Epoch 650 MSE = 0.582529
Epoch 700 MSE = 0.57376
Epoch 750 MSE = 0.566311
Epoch 800 MSE = 0.559985
Epoch 850 MSE = 0.554612
Epoch 900 MSE = 0.550048
Epoch 950 MSE = 0.546172


In [23]:
# minibatch gradient descent
tf.reset_default_graph()
y_reshape = housing.target.reshape([-1,1])
n_epochs = 1000
batch_size = 1000
n_batches = int(np.ceil(m/batch_size))
learning_rate = 0.01
X = tf.placeholder(tf.float32, shape=(None,n+1))
y = tf.placeholder(tf.float32, shape=(None,1))
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0))
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
def random_sampling(X,y,batch_size):
    random_indice = np.random.permutation(m)[:batch_size]
    return [X[random_indice], y[random_indice]]
def fetch_batch(X,y,batch_index, batch_size):
    from_i = batch_index*batch_size
    to_i = np.maximum((batch_index+1)*batch_size, X.shape[0])
    return [X[from_i:to_i], y[from_i:to_i]]
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(X_scaled_plus_bias, y_reshape, batch_index, batch_size)
            mse_, foo = sess.run([mse,training_op],feed_dict={X:X_batch,y:y_batch})
        if epoch % 50 == 0:
            print("Epoch", epoch, "MSE =", mse_)
    best_theta = theta.eval()

Epoch 0 MSE = 2.29027
Epoch 50 MSE = 0.406323
Epoch 100 MSE = 0.390892
Epoch 150 MSE = 0.387513
Epoch 200 MSE = 0.386357
Epoch 250 MSE = 0.385931
Epoch 300 MSE = 0.385773
Epoch 350 MSE = 0.385715
Epoch 400 MSE = 0.385693
Epoch 450 MSE = 0.385685
Epoch 500 MSE = 0.385682
Epoch 550 MSE = 0.385681
Epoch 600 MSE = 0.38568
Epoch 650 MSE = 0.38568
Epoch 700 MSE = 0.38568
Epoch 750 MSE = 0.38568
Epoch 800 MSE = 0.38568
Epoch 850 MSE = 0.38568
Epoch 900 MSE = 0.38568
Epoch 950 MSE = 0.38568


In [21]:
def random_sampling(X,y,batch_size):
    random_indice = np.random.permutation(m)[:batch_size]
    return [X[random_indice], y[random_indice]]
def fetch_batch(X,y,batch_index, batch_size):
    from_i = batch_index*batch_size
    to_i = np.maximum((batch_index+1)*batch_size, X.shape[0])
    return [X[from_i:to_i], y[from_i:to_i]]

In [34]:
print(m,n)

20640 8


In [32]:
# visualize
now = datetime.utcnow().strftime("%y%m%d%H%M%S")
root_logdir = "logs"
logdir = "{}/run-{}".format(root_logdir, now)
tf.reset_default_graph()
y_reshape = housing.target.reshape([-1,1])
n_epochs = 500
batch_size = 1000
n_batches = int(np.ceil(m/batch_size))
learning_rate = 0.01
X = tf.placeholder(tf.float32, shape=(None,n+1))
y = tf.placeholder(tf.float32, shape=(None,1))
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0))
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar("MSE",mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
def random_sampling(X,y,batch_size):
    random_indice = np.random.permutation(m)[:batch_size]
    return [X[random_indice], y[random_indice]]
def fetch_batch(X,y,batch_index, batch_size):
    from_i = batch_index*batch_size
    to_i = np.maximum((batch_index+1)*batch_size, X.shape[0])
    return [X[from_i:to_i], y[from_i:to_i]]
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if(epoch%100==0): print(epoch)
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(X_scaled_plus_bias, y_reshape, batch_index, batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
            if batch_index % 10 == 0:
                summary_str = sess.run(mse_summary, feed_dict={X:X_batch, y:y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
    best_theta = theta.eval()
    file_writer.close()

0
100
200
300
400


In [39]:
print(datetime.utcnow().strftime("%Y%M%D%H%M%S"))
print(datetime.utcnow().strftime("%y%m%d%H%M%S"))

20184804/14/18184807
180414184807
